## Sequence GAN from char-rnn
This is a character-level language model using recurrent neural networks based Sequence GAN (SeqGAN).
SeqGAN was proposed to cover discrete sequence data.
In this assignment, you will implement SeqGAN with shakespeare data used in assignment 3.

Original blog post & code:
https://github.com/LantaoYu/SeqGAN

That said, you are allowed to copy paste the codes from the original repo with an additional effort to apply it to our data.
HOWEVER, try to implement the model yourself first, and consider the original source code as a last resort.
You will learn a lot while wrapping around your head during the implementation. And you will understand more clearly in a code level.

### AND MOST IMPORTANTLY, IF YOU JUST BLINDLY COPY PASTE THE CODE, YOU SHALL RUIN YOUR EXAM.
### The exam is designed to be solvable for students that actually have written the code themselves.
At least strictly re-type the codes from the original repo line-by-line, and understand what each line means thoroughly.

## YOU HAVE BEEN WARNED.

Now proceed to the code. You may use textloader in previous assingment or not. You can freely create another python files (\*.py) and then import them. Following codes can be modified as you want. Just make sure that SeqGAN training works.



In [1]:
# ipython magic function for limiting the gpu to be seen for tensorflow
# if you have just 1 GPU, specify the value to 0
# if you have multiple GPUs (nut) and want to specify which GPU to use, specify this value to 0 or 1 or etc.
%env CUDA_DEVICE_ORDER = PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES = 2
# load a bunch of libraries
#from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.contrib import legacy_seq2seq
import numpy as np
import argparse
import time
import os
from six.moves import cPickle
from six import text_type
import sys

# this module is from the .py file of this folder
# it handles loading texts to digits (aka. tokens) which are recognizable for the model
from utils import TextLoader

# for TensorFlow vram efficiency: if this is not specified, the model hogs all the VRAM even if it's not necessary
# bad & greedy TF! but it has a reason for this design choice FWIW, try googling it if interested
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

data_dir = 'data/tinyshakespeare'
seq_length = 20
batch_size = 32

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


Write down the generator class and any other methods required for the generator class. You may define such methods in other python files (ex : utils.py).

In [2]:
#In the Model.py, there is the generator method.
import model
#We refered and copied coded from the original blog post & code(https://github.com/LantaoYu/SeqGAN and https://github.com/ofirnachum)
#Also, we applied the our things to the reference code.

Write down the discriminator class and any other methods required for the discriminator class. You may define such methods in other python files (ex : utils.py).

In [3]:
#In the Model.py, there is the discriminator, roll out, and other methods that helps to implement.
#We refered and copied some codes from the original blog post & code(https://github.com/LantaoYu/SeqGAN and https://github.com/ofirnachum)

If you need any other class or method, use below blanks. You may insert or delete blanks as many you want. Of course, you may define them in other python files and then import them.

In [4]:
from __future__ import print_function

import codecs

__doc__ = """Char-based Seq-GAN on data from a book."""


import train
from train import *
import os.path
import numpy as np
import tensorflow as tf
import random
import subprocess
import gzip


EMB_DIM = 20
HIDDEN_DIM = 25
SEQ_LENGTH = 20
START_TOKEN = 0

EPOCH_ITER = 1000
CURRICULUM_RATE = 0.02  # how quickly to move from supervised training to unsupervised
TRAIN_ITER = 300000  # generator/discriminator alternating
D_STEPS = 2  # how many times to train the discriminator per generator step
SEED = 88

DATA_FILE = 'data/tinyshakespeare/input.txt'


def tokenize(s):
    return [c for c in ' '.join(s.split())]


def get_data(download=not os.path.exists(DATA_FILE)):
    token_stream = []
    is_gzip = False
    try:
        open(DATA_FILE).read(2)
    except UnicodeDecodeError:
        print("HERE")
    with gzip.open(DATA_FILE) if is_gzip else codecs.open(DATA_FILE, 'r', 'utf-8',errors='ignore') as f:
        for line in f:
            line = line if not is_gzip else line.decode('utf-8')
            if ("Even to the court, the heart, to the seat o' the brain;" in line or token_stream) and line.strip():
                token_stream.extend(tokenize(line.strip().lower()))
                token_stream.append(' ')
            if len(token_stream) > 10000 * SEQ_LENGTH:  # enough data
                break

    return token_stream


class BookGRU(model.GRU):

    def d_optimizer(self, *args, **kwargs):
        return tf.train.AdamOptimizer()  # ignore learning rate

    def g_optimizer(self, *args, **kwargs):
        return tf.train.AdamOptimizer()  # ignore learning rate


def get_trainable_model(num_emb):
    return BookGRU(
        num_emb, EMB_DIM, HIDDEN_DIM,
        SEQ_LENGTH, START_TOKEN)


def get_random_sequence(token_stream, word2idx):
    """Returns random subsequence."""
    start_idx = random.randint(0, len(token_stream) - SEQ_LENGTH)
    return [word2idx[tok] for tok in token_stream[start_idx:start_idx + SEQ_LENGTH]]


def verify_sequence(three_grams, seq):
    """Not a true verification; only checks 3-grams."""
    for i in range(len(seq) - 3):
        if tuple(seq[i:i + 3]) not in three_grams:
            return False
    return True


def main():
    random.seed(SEED)
    np.random.seed(SEED)

    token_stream = get_data()
    assert START_TOKEN == 0
    words = ['_START'] + list(set(token_stream))
    word2idx = dict((word, i) for i, word in enumerate(words))
    num_words = len(words)
    three_grams = dict((tuple(word2idx[w] for w in token_stream[i:i + 3]), True)
                       for i in range(len(token_stream) - 3))
    extends_words = []
    print('num words', num_words)
    print('stream length', len(token_stream))
    print('distinct 3-grams', len(three_grams))

    trainable_model = get_trainable_model(num_words)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    print('training')
    for epoch in range(TRAIN_ITER // EPOCH_ITER):
        print(" ")
        print('epoch', epoch)
        proportion_supervised = max(0.0, 1.0 - CURRICULUM_RATE * epoch)
        train.train_epoch(
            sess, trainable_model, EPOCH_ITER,
            proportion_supervised=proportion_supervised,
            g_steps=1, d_steps=D_STEPS,
            next_sequence=lambda: get_random_sequence(token_stream, word2idx),
            verify_sequence=lambda seq: verify_sequence(three_grams, seq),
            words=words)
    #print('words will be like ', words)
    print(*extends_words, sep = "\n") 

if __name__ == '__main__':
    main()

num words 37
stream length 200018
distinct 3-grams 5193
training
 
epoch 0
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 1.00 will be supervised
None
 
 
epoch 1
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.98 will be supervised
['g', 't', 'w', 't', 'm', 'o', 'd', ' ', 't', 'n', 'h', ' ', 't', 'y', 'e', '.', ':', 'h', ' ', 'i']
 
epoch 2
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.96 will be supervised
['g', 'h', 'i', 'd', ' ', 't', 'e', 'a', 'u', 'a', 'h', 't', 'e', '.', ' ', 'w', ' ', 'a', 'r', ' ']
 
epoch 3
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.94 will be supervised
[',', '.', ' ', 't', 'i', 't', ' ', 'r', 'h', 'l', ' ', 't', 'e', 'u', 'n', 'm', 'e', 'w', 'r', ' ']
 
epoch 4
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.92 will be supervised
['h', 'y', ' ', 'b', 'o', 't', 'e', 'd', 'e', 'n', 'n', 'i', 'n', 'o', ' ', 's', 'v', 'h', ' ', 'n']
 
epoch 5

[' ', 'm', 'a', 'm', 'd', 'e', 's', ' ', 'p', 'e', 'r', 'r', 'i', 'c', 'e', 'r', ' ', 'd', 'a', ',']
 
epoch 41
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.18 will be supervised
['n', 'o', 'n', ';', ' ', 'n', 'e', 's', 'l', 'e', 's', ' ', 't', 'h', 'y', ',', ' ', 't', 'h', 'e']
 
epoch 42
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.16 will be supervised
['w', 'e', ' ', 'p', 'l', 'e', ' ', 'w', 'l', 'e', 't', ' ', 'v', 'i', 's', ' ', 't', 'o', ' ', 'm']
 
epoch 43
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.14 will be supervised
['t', '?', ' ', 'h', 'e', ' ', 'm', 'e', ' ', 'a', 'n', 'l', ' ', 'b', 'i', 'l', 'l', ' ', 'i', ' ']
 
epoch 44
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.12 will be supervised
['e', 'l', 'a', 'm', ' ', 'h', 'e', ',', ' ', 's', 'h', 'e', 'e', ' ', 'a', 'm', 'i', 'n', 'g', 'i']
 
epoch 45
running 1000 iterations with 1 g steps and 2 d steps
of the g steps,

['u', 'l', 'y', 's', 'e', ' ', 'c', 'a', 'a', 'n', 'd', ' ', 'c', 'o', 'r', 'd', 'e', ' ', 'c', 'o']
 
epoch 81
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['o', 'u', 's', ';', ' ', 't', 'h', 'i', 't', 'h', 'i', 'r', 's', ' ', 'e', ' ', 'n', 'u', 's', ' ']
 
epoch 82
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['e', 'n', 'y', ',', 'e', ' ', 'm', 'e', 'r', 'y', ' ', 'o', 'l', ' ', 's', 'o', 'm', 't', '!', ' ']
 
epoch 83
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['r', 'c', 'e', 'a', 'r', 'g', ' ', 'w', 'a', 'l', 'l', ' ', 'f', 'o', 'n', ' ', 'b', 'a', 'n', 'o']
 
epoch 84
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['o', 'u', 'n', '!', ' ', 'w', 'i', 't', 'h', 'e', ' ', 'w', 'e', ' ', 'b', 'o', 'u', 's', 'e', 's']
 
epoch 85
running 1000 iterations with 1 g steps and 2 d steps
of the g steps,

[' ', 's', 'e', 'y', 'e', 'n', 'g', 'o', ' ', 's', 'o', 'u', 'l', 'd', ' ', 'b', 'u', 's', 'e', 'r']
 
epoch 121
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['n', 'e', ' ', 'o', 'x', 's', 'i', 'l', 'd', ' ', 'k', 'a', 'r', 'e', ' ', 'n', 'i', 'c', 'e', "'"]
 
epoch 122
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['t', ' ', 'f', 'u', 'd', 'e', 'n', 'c', 'e', 'n', 'c', 'e', 'r', ' ', 'f', 'o', 'r', 'e', ' ', 't']
 
epoch 123
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['o', 'f', ' ', 'k', 'a', 'r', ',', ' ', 'i', 't', ' ', 'c', 'e', ',', ' ', 'g', 'i', 'p', 'l', 'o']
 
epoch 124
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['l', 'y', ' ', 'f', 'o', 'r', 'y', ' ', 'f', 'o', 'l', 'm', ' ', 'i', 'n', ' ', 'p', 'e', 'l', 'l']
 
epoch 125
running 1000 iterations with 1 g steps and 2 d steps
of the g s

['o', 'u', ' ', 'o', 'n', 'e', ' ', 't', 'h', 'e', 'y', ' ', 'n', 'o', 't', ' ', 'h', 'a', 't', 'h']
 
epoch 161
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['e', ' ', 't', 'h', 'e', ' ', 's', 'c', 'h', 'o', ',', ' ', 'a', 's', ' ', 'a', ' ', 's', 'f', 'a']
 
epoch 162
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['s', ' ', 'f', 'a', 'e', 'r', 'p', 'a', 'o', 'w', ' ', 'i', 'n', ' ', 'm', 'u', 'm', 'y', '-', 'b']
 
epoch 163
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
[' ', 'y', 'o', 'u', ' ', 'w', 'o', 'l', ' ', 'b', 'i', 'v', 'e', ',', ' ', 't', 'h', 'e', ' ', 't']
 
epoch 164
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['t', 'i', 't', 'o', 'm', 'e', ' ', 'a', 'n', 'd', ' ', 'a', 'n', 'd', ' ', 'h', 'i', 's', ' ', 'i']
 
epoch 165
running 1000 iterations with 1 g steps and 2 d steps
of the g s

['a', 'r', ',', ' ', 'l', 'o', 'k', 'i', 'n', 'g', ',', ' ', 'r', 'o', 'm', 'e', ' ', 'a', 'n', 'g']
 
epoch 201
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['f', 'i', 't', 's', 'a', 'v', 'u', 'n', 't', 'e', 'r', 'i', 'v', 'e', ',', ' ', 'c', 'o', 'n', 'd']
 
epoch 202
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['i', ' ', 't', 'h', 'a', 't', ' ', 'b', 'u', 't', '?', ' ', 'h', 'e', 'a', 'r', 'd', ' ', 'r', 'o']
 
epoch 203
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['i', 't', ' ', 'i', 't', 'a', 'o', 'l', 'd', ' ', 'm', 'a', 'l', 'l', ' ', 'o', 'f', ' ', 'm', 'a']
 
epoch 204
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['f', 'l', ',', ' ', 'a', 'r', 'e', ' ', 'a', ' ', 'i', ' ', 't', 'h', 'e', 'f', ' ', 'y', 'o', 'u']
 
epoch 205
running 1000 iterations with 1 g steps and 2 d steps
of the g s

['h', ' ', 'm', 'e', ' ', 'y', 'o', 'u', 'r', ' ', 'o', 'f', ' ', 't', 'o', ' ', 'b', 'i', 'n', 'g']
 
epoch 241
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
[' ', 'w', 'h', 'e', 'r', 'd', ' ', 'a', 's', ' ', 'i', 't', ' ', 'p', 'a', 'd', 'd', ' ', 'r', 'e']
 
epoch 242
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
[' ', 'm', 'y', '?', ' ', 'l', 'o', 'n', 'd', ',', ',', ' ', 'y', 'o', 'u', ' ', 'y', 'r', 'o', ' ']
 
epoch 243
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['s', ',', ' ', 'a', 't', ' ', 'm', 'y', ' ', 'h', 'a', 'r', 't', ' ', 'p', 'u', 'i', 'i', 's', 'a']
 
epoch 244
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['e', 'n', 'c', 'e', ' ', 't', 'h', 'a', 't', ' ', 'g', 'e', 'a', "'", 's', ' ', 'b', 'o', 'w', ' ']
 
epoch 245
running 1000 iterations with 1 g steps and 2 d steps
of the g s

['u', ' ', 't', 'i', ' ', 'i', 't', 't', "'", 'l', ' ', 'a', 't', ' ', 'a', 'r', 'e', 'd', '.', ' ']
 
epoch 281
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['t', 'a', 'r', 's', ' ', 't', 'h', 'o', 'u', '?', ' ', 'v', 'a', 'r', 's', 'e', 'l', 'd', ' ', 'a']
 
epoch 282
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['s', ' ', 'l', 'a', 'c', "'", 'r', 'a', 'r', 'n', 'o', 'v', 'e', 'n', ' ', 'h', 'a', 'n', ' ', 'r']
 
epoch 283
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
[' ', 't', 'h', 'o', 'k', ' ', 'e', 'n', 'n', 'l', 's', 'i', 'n', 'i', 'e', 's', ':', ' ', 't', 'i']
 
epoch 284
running 1000 iterations with 1 g steps and 2 d steps
of the g steps, 0.00 will be supervised
['h', ' ', 'm', 'u', 'l', ' ', 'l', 'e', ' ', 'w', 'i', 'l', 'l', ':', ' ', 'g', 'e', ' ', 'i', 'n']
 
epoch 285
running 1000 iterations with 1 g steps and 2 d steps
of the g s

In [5]:
print(extends_words)

['gtwtmod tnh tye.:h i', 'ghid teauahte. w ar ', ',. tit rhl teunmewr ', 'hy botedennino svh n', 's cht litailthotreot', 'eiipivelonu!: ecisrl', ' dapritin uithe anri', "oweg: h woy dhao'une", ', ras p! aargin hoiy', 'end timo l vivot pot', 'evers: he i,om. onto', 'loll thor, bens nbba', 'at meey shof met ace', 's i v hee ti the het', " the yosn'lete tony ", 'he.: ivos cire. ther', "ate'tid thos? adu co", ' thy as oy sesmae: c', "ou-muey; rre, toro'c", 'erlerrt ne thoinu, t', ' deu: cof hinporea, ', '  cate this foder ea', 'l bemienit fattomiks', "od thiiwoes.ars:'s l", 'eim her thy mman sha', 'cews. veneus! hus: t', 'rrit,. us, iso hoerd', ' ther youblelle. wan', 'y it thelonor imdt. ', 'e s-mitwe: thherr, f', 'leks. a yon thoulans', 'tesce old ife mir a ', 'love on the thir ot ', 'hat in chius? hove i', ' ther ?oadswit canty', 'ch, u n so of ces fm', 'h be bfiiss wangloiu', 'bo maur miskoliceern', 'e ci beshireshe fuss', ' mamdes perricer da,', 'non; nesles thy, the', 'we ple wlet vi

In [7]:
print(*extends_words, sep = " ")

gtwtmod tnh tye.:h i ghid teauahte. w ar  ,. tit rhl teunmewr  hy botedennino svh n s cht litailthotreot eiipivelonu!: ecisrl  dapritin uithe anri oweg: h woy dhao'une , ras p! aargin hoiy end timo l vivot pot evers: he i,om. onto loll thor, bens nbba at meey shof met ace s i v hee ti the het  the yosn'lete tony  he.: ivos cire. ther ate'tid thos? adu co  thy as oy sesmae: c ou-muey; rre, toro'c erlerrt ne thoinu, t  deu: cof hinporea,    cate this foder ea l bemienit fattomiks od thiiwoes.ars:'s l eim her thy mman sha cews. veneus! hus: t rrit,. us, iso hoerd  ther youblelle. wan y it thelonor imdt.  e s-mitwe: thherr, f leks. a yon thoulans tesce old ife mir a  love on the thir ot  hat in chius? hove i  ther ?oadswit canty ch, u n so of ces fm h be bfiiss wangloiu bo maur miskoliceern e ci beshireshe fuss  mamdes perricer da, non; nesles thy, the we ple wlet vis to m t? he me anl bill i  elam he, shee amingi us wle focamer to iu hoam; gooe non chear uri nint mas thies t ou gheils: th

In [9]:
for i in range(32, 73):
    print(extends_words[i], end=' ')

love on the thir ot  hat in chius? hove i  ther ?oadswit canty ch, u n so of ces fm h be bfiiss wangloiu bo maur miskoliceern e ci beshireshe fuss  mamdes perricer da, non; nesles thy, the we ple wlet vis to m t? he me anl bill i  elam he, shee amingi us wle focamer to iu hoam; gooe non chear uri nint mas thies t ou gheils: there the ouudie anld son come -deimot dege mes, of amuy-thin that werk  os dor tit dond the  , is yumd hey dul th  rorveil ridun. wat  t the to culold gom  an nom jus the he me pwre volget? fand pe oun, hillivir, mort,  wrd wre beves, bes  des, thass andin man ves, sowher, wot, go  od; to pat rome't f a'ls her if ber wor  ibe sitheor jome lot  the an payoukre' an or, beaseh als gardg se heirt'thacathe ho oud the mamancingd t t se pider hon sargi his e' parpce githes e you durthithes: it m shend anucham puse liges ciges, the: wr 